# 10 - Engenharia de Features Avançada

Este notebook cria features avançadas baseadas nos dados de treino e validação temporal.

## Novas Features:
- **Target de Classificação**: `vendeu` (1 se quantidade > 0, 0 caso contrário)
- **Features de Preço**: preço unitário, variações, médias
- **Features de Calendário**: dia do mês, semana do mês, flags temporais
- **Features de Hierarquia**: agregações por categoria, zipcode
- **Features de Tendência**: momentum, aceleração de vendas

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import warnings
warnings.filterwarnings('ignore')

print('🚀 Iniciando Engenharia de Features Avançada')
print('🎯 Objetivo: Criar variáveis preditivas poderosas para o modelo de dois estágios')

# Criar pasta submissao3 se não existir
os.makedirs('../data/submissao3', exist_ok=True)
print('📁 Pasta data/submissao3 criada/verificada')

🚀 Iniciando Engenharia de Features Avançada
🎯 Objetivo: Criar variáveis preditivas poderosas para o modelo de dois estágios
📁 Pasta data/submissao3 criada/verificada


## 1. Carregamento dos Dados de Treino e Validação

In [2]:
# Carregar dados de treino e validação
print('📂 Carregando dados de treino e validação...')

train_data = pd.read_parquet('../data/submissao3/train_data.parquet')
validation_data = pd.read_parquet('../data/submissao3/validation_data.parquet')

print(f'🏋️ Dados de treino: {train_data.shape}')
print(f'🔍 Dados de validação: {validation_data.shape}')

# Verificar período
print(f'📅 Treino: {train_data["semana"].min()} até {train_data["semana"].max()}')
print(f'📅 Validação: {validation_data["semana"].min()} até {validation_data["semana"].max()}')

print('✅ Dados carregados com sucesso')

📂 Carregando dados de treino e validação...
🏋️ Dados de treino: (50126880, 6)
🔍 Dados de validação: (5221550, 6)
📅 Treino: 2021-12-28 00:00:00 até 2022-11-22 00:00:00
📅 Validação: 2022-11-29 00:00:00 até 2022-12-27 00:00:00
✅ Dados carregados com sucesso


## 2. Carregamento dos Dados Auxiliares

In [3]:
# Carregar dados de produtos e PDVs para features adicionais
print('📂 Carregando dados auxiliares...')

# Produtos
produtos = pd.read_parquet(
    '../data/part-00000-tid-7173294866425216458-eae53fbf-d19e-4130-ba74-78f96b9675f1-4-1-c000.snappy.parquet'
)
produtos = produtos.rename(columns={
    'produto': 'produto_id'
})

# PDVs
pdvs = pd.read_parquet(
    '../data/part-00000-tid-2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36-4-1-c000.snappy.parquet'
)
pdvs = pdvs.rename(columns={
    'pdv': 'pdv_id',
    'categoria_pdv': 'tipo_loja'
})

print(f'📊 Produtos: {produtos.shape} - Categorias: {produtos["categoria"].nunique()}')
print(f'📊 PDVs: {pdvs.shape} - Zipcodes: {pdvs["zipcode"].nunique()}')

print('✅ Dados auxiliares carregados')

📂 Carregando dados auxiliares...
📊 Produtos: (7092, 8) - Categorias: 7
📊 PDVs: (14419, 4) - Zipcodes: 788
✅ Dados auxiliares carregados


## 3. Função de Engenharia de Features

In [4]:
def criar_features_avancadas_otimizada(df, produtos_df, pdvs_df, nome_conjunto):
    """
    Criar features avançadas de forma OTIMIZADA e SEM TARGET LEAKAGE.
    Esta versão substitui o groupby().apply() por operações vetorizadas.
    """
    print(f'\n🔧 Criando features avançadas para {nome_conjunto} (VERSÃO OTIMIZADA)...')
    
    dados = df.copy()
    
    # ETAPAS INICIAIS (JÁ CORRETAS)
    print('   🎯 Criando target de classificação...')
    dados['vendeu'] = (dados['quantidade'] > 0).astype(int)
    
    print('   🔗 Fazendo merge com dados auxiliares...')
    dados = dados.merge(produtos_df[['produto_id', 'categoria']], on='produto_id', how='left')
    dados = dados.merge(pdvs_df[['pdv_id', 'zipcode', 'tipo_loja']], on='pdv_id', how='left')
    
    print('   🔄 Ordenando dados para consistência temporal...')
    dados.sort_values(['pdv_id', 'produto_id', 'semana'], inplace=True)
    dados.reset_index(drop=True, inplace=True)
    
    gb_cols = ['pdv_id', 'produto_id']
    
    # ============================================================================
    # REFAZENDO AS FEATURES DE FORMA OTIMIZADA
    # ============================================================================
    
    # 1. FEATURES DE PREÇO E LAGS (SEM LEAKAGE)
    print('   💰 Criando features de preço e lags (otimizado)...')
    preco_unitario_temp = np.where(dados['quantidade'] > 0, dados['faturamento'] / dados['quantidade'], 0)
    
    gb_preco = dados.assign(preco_unitario_temp=preco_unitario_temp).groupby(gb_cols)['preco_unitario_temp']
    dados['preco_lag_1'] = gb_preco.shift(1)
    dados['preco_lag_2'] = gb_preco.shift(2)
    dados['variacao_preco_sku_semanal'] = (dados['preco_lag_1'] - dados['preco_lag_2']).fillna(0)
    
    # 2. FEATURES DE LAG E ROLLING (OTIMIZAÇÃO CRÍTICA)
    print('   📊 Criando features de lag e rolling (OTIMIZAÇÃO CRÍTICA)...')
    
    # Agrupar uma única vez para performance
    gb_quantidade = dados.groupby(gb_cols)['quantidade']
    
    # Lags de quantidade
    for lag in [1, 2, 3, 4]:
        dados[f'quantidade_lag_{lag}'] = gb_quantidade.shift(lag)
        
    # --- A MUDANÇA MAIS IMPORTANTE ESTÁ AQUI ---
    # Substituindo o groupby().apply() lento por groupby().rolling() otimizado
    rolling_window = gb_quantidade.rolling(window=4, min_periods=1)
    
    dados['quantidade_media_4w'] = rolling_window.mean().reset_index(level=gb_cols, drop=True)
    dados['quantidade_std_4w'] = rolling_window.std().reset_index(level=gb_cols, drop=True)
    dados['quantidade_max_4w'] = rolling_window.max().reset_index(level=gb_cols, drop=True)
    # --- FIM DA MUDANÇA CRÍTICA ---

    # 3. DEMAIS FEATURES (JÁ ERAM RELATIVAMENTE EFICIENTES, MAS PODEM USAR OS RESULTADOS OTIMIZADOS)
    print('   📈 Criando features de tendência e hierarquia...')
    
    # Hierarquia (usando lags já calculados)
    dados['media_vendas_categoria_pdv_lag_1'] = dados.groupby(['semana', 'categoria', 'pdv_id'])['quantidade_lag_1'].transform('mean')
    vendas_categoria_pdv_lag = dados.groupby(['semana', 'categoria', 'pdv_id'])['quantidade_lag_1'].transform('sum')
    dados['share_vendas_sku_categoria_lag_1'] = (dados['quantidade_lag_1'] / vendas_categoria_pdv_lag).fillna(0)

    # Tendência (usando lags já calculados)
    dados['momentum_ratio'] = (dados['quantidade_lag_1'] / dados['quantidade_media_4w']).fillna(0)
    dados['aceleracao'] = (dados['quantidade_lag_1'] - dados['quantidade_lag_2']).fillna(0)
    
    # 4. FEATURES DE CALENDÁRIO (SEMPRE SEGURAS)
    print('   📅 Criando features de calendário...')
    dados['dia_do_mes'] = dados['semana'].dt.day
    dados['semana_do_mes'] = (dados['semana'].dt.day - 1) // 7 + 1
    dados['eh_inicio_mes'] = (dados['semana'].dt.day <= 7).astype(int)
    dados['eh_fim_mes'] = (dados['semana'].dt.day >= 22).astype(int)
    dados['mes'] = dados['semana'].dt.month
    
    # Features cíclicas para mês
    dados['mes_sin'] = np.sin(2 * np.pi * dados['mes'] / 12)
    dados['mes_cos'] = np.cos(2 * np.pi * dados['mes'] / 12)

    # 5. FEATURES PARA REGRESSÃO (usando valores atuais)
    print('   🎯 Criando features específicas para REGRESSÃO...')
    dados['preco_unitario_atual'] = preco_unitario_temp 
    dados['preco_medio_semanal_sku_atual'] = dados.groupby(['semana', 'produto_id'])['preco_unitario_atual'].transform('mean')
    dados['media_vendas_categoria_pdv_atual'] = dados.groupby(['semana', 'categoria', 'pdv_id'])['quantidade'].transform('mean')
    vendas_categoria_pdv_atual = dados.groupby(['semana', 'categoria', 'pdv_id'])['quantidade'].transform('sum')
    dados['share_vendas_sku_categoria_atual'] = (dados['quantidade'] / vendas_categoria_pdv_atual).fillna(0)
    
    # 6. FEATURES CATEGÓRICAS HASH
    print('   🏷️ Criando features categóricas...')
    dados['pdv_hash'] = dados['pdv_id'].astype(str).apply(hash).abs() % 100
    dados['produto_hash'] = dados['produto_id'].astype(str).apply(hash).abs() % 100
    dados['categoria_hash'] = dados['categoria'].astype(str).apply(hash).abs() % 50
    dados['zipcode_hash'] = dados['zipcode'].astype(str).apply(hash).abs() % 1000
    
    # Features de interação
    dados['pdv_produto_hash'] = dados['pdv_hash'] * 100 + dados['produto_hash']
    dados['categoria_zipcode_hash'] = dados['categoria_hash'] * 1000 + dados['zipcode_hash']
    
    # 7. LIMPEZA FINAL
    print('   🧹 Aplicando limpeza final...')
    dados.fillna(0, inplace=True)
    
    print(f'   ✅ {nome_conjunto} - Features criadas com sucesso: {dados.shape}')
    return dados

print('✅ Função de engenharia de features OTIMIZADA definida')

✅ Função de engenharia de features OTIMIZADA definida


## 4. Aplicação da Engenharia de Features

In [5]:
# Aplicar engenharia de features OTIMIZADA aos dois conjuntos
print('🔧 Aplicando engenharia de features OTIMIZADA...')

train_features = criar_features_avancadas_otimizada(train_data, produtos, pdvs, "TREINO")
validation_features = criar_features_avancadas_otimizada(validation_data, produtos, pdvs, "VALIDAÇÃO")

print('\n✅ Engenharia de features OTIMIZADA aplicada a ambos os conjuntos!')

🔧 Aplicando engenharia de features OTIMIZADA...

🔧 Criando features avançadas para TREINO (VERSÃO OTIMIZADA)...
   🎯 Criando target de classificação...
   🔗 Fazendo merge com dados auxiliares...
   🔄 Ordenando dados para consistência temporal...
   💰 Criando features de preço e lags (otimizado)...
   📊 Criando features de lag e rolling (OTIMIZAÇÃO CRÍTICA)...
   📈 Criando features de tendência e hierarquia...
   📅 Criando features de calendário...
   🎯 Criando features específicas para REGRESSÃO...
   🏷️ Criando features categóricas...
   🧹 Aplicando limpeza final...
   ✅ TREINO - Features criadas com sucesso: (50126880, 41)

🔧 Criando features avançadas para VALIDAÇÃO (VERSÃO OTIMIZADA)...
   🎯 Criando target de classificação...
   🔗 Fazendo merge com dados auxiliares...
   🔄 Ordenando dados para consistência temporal...
   💰 Criando features de preço e lags (otimizado)...
   📊 Criando features de lag e rolling (OTIMIZAÇÃO CRÍTICA)...
   📈 Criando features de tendência e hierarquia...

## 5. Análise das Features Criadas

In [6]:
# Analisar features criadas SEM TARGET LEAKAGE
print('📊 Analisando features criadas SEM TARGET LEAKAGE...')

# Listar todas as features
features_disponiveis = list(train_features.columns)
print(f'\n📋 Features disponíveis ({len(features_disponiveis)}):')

# Categorizar features CORRIGIDAS
features_categorias = {
    'Básicas': ['semana', 'pdv_id', 'produto_id', 'quantidade', 'faturamento', 'distributor_id'],
    'Target': ['vendeu'],
    'Auxiliares': ['categoria', 'zipcode', 'tipo_loja'],
    'Preço_SEM_Leakage': [col for col in features_disponiveis if 'preco' in col and 'lag' in col],
    'Preço_PARA_Regressão': [col for col in features_disponiveis if 'preco' in col and 'atual' in col],
    'Calendário': [col for col in features_disponiveis if any(x in col for x in ['dia', 'semana_do', 'mes', 'inicio', 'fim'])],
    'Lag': [col for col in features_disponiveis if 'lag_' in col and 'preco' not in col],
    'Rolling': [col for col in features_disponiveis if any(x in col for x in ['media_4w', 'std_4w', 'max_4w'])],
    'Hierarquia_SEM_Leakage': [col for col in features_disponiveis if any(x in col for x in ['media_vendas', 'share']) and 'lag' in col],
    'Hierarquia_PARA_Regressão': [col for col in features_disponiveis if any(x in col for x in ['media_vendas', 'share']) and 'atual' in col],
    'Tendência': [col for col in features_disponiveis if any(x in col for x in ['momentum', 'aceleracao'])],
    'Hash': [col for col in features_disponiveis if 'hash' in col]
}

for categoria, features in features_categorias.items():
    if features:
        print(f'\n🏷️ {categoria} ({len(features)}):')
        for feat in features:
            if feat in features_disponiveis:
                print(f'   • {feat}')

print('\n🚨 ANÁLISE DE TARGET LEAKAGE:')
features_sem_leakage = []
features_com_leakage = []

for feat in features_disponiveis:
    if feat in ['semana', 'pdv_id', 'produto_id', 'quantidade', 'faturamento', 'distributor_id', 'vendeu']:
        continue  # Básicas, não contar
    elif 'atual' in feat:
        features_com_leakage.append(feat)  # Para regressão apenas
    else:
        features_sem_leakage.append(feat)  # Seguras para classificação

print(f'✅ Features SEGURAS para classificação: {len(features_sem_leakage)}')
for feat in features_sem_leakage[:10]:  # Mostrar apenas primeiras 10
    print(f'   • {feat}')
if len(features_sem_leakage) > 10:
    print(f'   • ... e mais {len(features_sem_leakage)-10} features')

print(f'\n⚠️ Features COM LEAKAGE (apenas para regressão): {len(features_com_leakage)}')
for feat in features_com_leakage:
    print(f'   • {feat}')

print('\n📊 Estatísticas do conjunto de treino (SEM LEAKAGE):')
print(f'   • Shape: {train_features.shape}')
print(f'   • Target vendeu: {train_features["vendeu"].sum():,} positivos ({train_features["vendeu"].mean()*100:.1f}%)')
print(f'   • Quantidade média: {train_features["quantidade"].mean():.2f}')
print(f'   • Registros com lag_4: {(train_features["quantidade_lag_4"] > 0).sum():,}')

print('\n📊 Estatísticas do conjunto de validação (SEM LEAKAGE):')
print(f'   • Shape: {validation_features.shape}')
print(f'   • Target vendeu: {validation_features["vendeu"].sum():,} positivos ({validation_features["vendeu"].mean()*100:.1f}%)')
print(f'   • Quantidade média: {validation_features["quantidade"].mean():.2f}')
print(f'   • Registros com lag_4: {(validation_features["quantidade_lag_4"] > 0).sum():,}')

print('✅ Análise das features SEM TARGET LEAKAGE concluída')

📊 Analisando features criadas SEM TARGET LEAKAGE...

📋 Features disponíveis (41):

🏷️ Básicas (6):
   • semana
   • pdv_id
   • produto_id
   • quantidade
   • faturamento
   • distributor_id

🏷️ Target (1):
   • vendeu

🏷️ Auxiliares (3):
   • categoria
   • zipcode
   • tipo_loja

🏷️ Preço_SEM_Leakage (2):
   • preco_lag_1
   • preco_lag_2

🏷️ Preço_PARA_Regressão (2):
   • preco_unitario_atual
   • preco_medio_semanal_sku_atual

🏷️ Calendário (10):
   • quantidade_media_4w
   • media_vendas_categoria_pdv_lag_1
   • dia_do_mes
   • semana_do_mes
   • eh_inicio_mes
   • eh_fim_mes
   • mes
   • mes_sin
   • mes_cos
   • media_vendas_categoria_pdv_atual

🏷️ Lag (6):
   • quantidade_lag_1
   • quantidade_lag_2
   • quantidade_lag_3
   • quantidade_lag_4
   • media_vendas_categoria_pdv_lag_1
   • share_vendas_sku_categoria_lag_1

🏷️ Rolling (3):
   • quantidade_media_4w
   • quantidade_std_4w
   • quantidade_max_4w

🏷️ Hierarquia_SEM_Leakage (2):
   • media_vendas_categoria_pdv_lag_1
   

## 6. Salvamento dos Dados Enriquecidos

In [7]:
# Salvar datasets enriquecidos
print('💾 Salvando datasets enriquecidos...')

# Função para limpar tipos de dados antes de salvar
def limpar_tipos_dados(df):
    """Limpa tipos de dados problemáticos para salvar em Parquet"""
    df_clean = df.copy()
    
    # Converter colunas object que são numéricas
    for col in df_clean.columns:
        if df_clean[col].dtype == 'object':
            try:
                # Tentar converter para numeric
                df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
                # Se ainda tem NaN, preencher com 0
                if df_clean[col].isna().any():
                    df_clean[col] = df_clean[col].fillna(0)
            except:
                # Se não conseguir converter, manter como string
                df_clean[col] = df_clean[col].astype(str)
    
    # Garantir que colunas ID sejam inteiras
    id_cols = ['pdv_id', 'produto_id', 'distributor_id']
    for col in id_cols:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].astype('int64')
    
    # Garantir que colunas categóricas sejam string
    cat_cols = ['categoria', 'zipcode', 'tipo_loja']
    for col in cat_cols:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].astype(str)
    
    return df_clean

# Limpar tipos dos datasets
print('🧹 Limpando tipos de dados...')
train_features_clean = limpar_tipos_dados(train_features)
validation_features_clean = limpar_tipos_dados(validation_features)

# Salvar em data/submissao3
train_features_clean.to_parquet('../data/submissao3/train_features.parquet', index=False)
validation_features_clean.to_parquet('../data/submissao3/validation_features.parquet', index=False)

print('✅ Arquivos salvos:')
print('   • data/submissao3/train_features.parquet')
print('   • data/submissao3/validation_features.parquet')

# Salvar metadados das features
import pickle

metadados_features = {
    'data_criacao': pd.Timestamp.now(),
    'estrategia': 'Engenharia de Features Avançada para Modelo de Dois Estágios',
    'total_features': len(train_features_clean.columns),
    'features_por_categoria': {cat: len(feats) for cat, feats in features_categorias.items() if feats},
    'target_classificacao': 'vendeu (1 se quantidade > 0, 0 caso contrário)',
    'target_regressao': 'quantidade (apenas onde vendeu = 1)',
    'registros_treino': len(train_features_clean),
    'registros_validacao': len(validation_features_clean),
    'features_lista': list(train_features_clean.columns),
    'observacoes': [
        'Features de preço incluem variações e médias temporais',
        'Features de calendário incluem componentes cíclicos',
        'Features de hierarquia agregam por categoria e zipcode',
        'Features de tendência capturam momentum e aceleração',
        'Features hash criam embeddings para variáveis categóricas'
    ]
}

with open('../data/submissao3/advanced_features_metadata.pkl', 'wb') as f:
    pickle.dump(metadados_features, f)

print('📋 Metadados salvos em: data/submissao3/advanced_features_metadata.pkl')

print('\n🎉 ENGENHARIA DE FEATURES AVANÇADA CONCLUÍDA!')
print('=' * 70)
print('🎯 Features criadas para modelo de dois estágios:')
print('   📊 CLASSIFICAÇÃO: Use target "vendeu" para treinar se vai vender')
print('   📊 REGRESSÃO: Use target "quantidade" (apenas onde vendeu=1) para quanto vai vender')
print('\n💡 Próximos passos:')
print('   1. Treinar modelo de classificação (LGBMClassifier)')
print('   2. Treinar modelo de regressão (LGBMRegressor)')
print('   3. Combinar os dois no pipeline final')
print('\n🚀 Dados prontos para modelagem avançada!')

💾 Salvando datasets enriquecidos...
🧹 Limpando tipos de dados...
✅ Arquivos salvos:
   • data/submissao3/train_features.parquet
   • data/submissao3/validation_features.parquet
📋 Metadados salvos em: data/submissao3/advanced_features_metadata.pkl

🎉 ENGENHARIA DE FEATURES AVANÇADA CONCLUÍDA!
🎯 Features criadas para modelo de dois estágios:
   📊 CLASSIFICAÇÃO: Use target "vendeu" para treinar se vai vender
   📊 REGRESSÃO: Use target "quantidade" (apenas onde vendeu=1) para quanto vai vender

💡 Próximos passos:
   1. Treinar modelo de classificação (LGBMClassifier)
   2. Treinar modelo de regressão (LGBMRegressor)
   3. Combinar os dois no pipeline final

🚀 Dados prontos para modelagem avançada!
